# Feature Engineering

In this notebook, we generate additional features that will help us come up with a more predictive model.

In [1]:
# Import necessary libraries
import pandas as pd
import dill
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
# Read unprocessed data (old and new datasets already merged)
df_xunprocessed=dill.load(open('./data/NYC-features.dill','rb'))
#df_unprocessed = dill.load(open('./data/NYC-finalfeatures.dill','rb'))
df_reviews = dill.load(open('./data/NYC-google-reviews1.dill','rb'))

In [4]:
len(df_xunprocessed)

16517

In [25]:
df_reviews.head()

,business_id,text,user_id,relative_time_description,time_created,user_rating
0,RDE8pT0YZbz4BsO_sPKbcg,Great place for lunch! Pleasant atmosphere. Am...,Daniel Donnelly,2 months ago,2019-02-26 02:49:43,5
1,RDE8pT0YZbz4BsO_sPKbcg,It’s an amazing little coffee shop. The food ...,Valentina Hernandez,2 weeks ago,2019-04-11 19:31:10,5
2,RDE8pT0YZbz4BsO_sPKbcg,"Awesome coffee, delicious pastries, and an inc...",Patrick Hart,2 months ago,2019-02-19 17:40:37,5
3,RDE8pT0YZbz4BsO_sPKbcg,Gem in the Bronx! Their food and drinks are am...,Ange Mallonga,2 weeks ago,2019-04-15 17:20:13,5
4,RDE8pT0YZbz4BsO_sPKbcg,This place says “gentrification” all over it. ...,Nick Mennell,5 months ago,2018-11-29 15:50:10,5


Let's list all the columns to think about our options for feature engineering.

In [26]:
df_xunprocessed.columns

Index(['business_id', 'categories', 'city', 'user_ratings_total',
       'coordinates.latitude', 'coordinates.longitude', 'name',
       'display_phone', 'location.display_address', 'location.state',
       'location.zipcode', 'is_closed', 'rating', 'review_count', 'price',
       'transcations', 'url'],
      dtype='object')

Each encrypted business_id from the df_unprocessed dataframe has corresponding entries in the df_reviews dataframe with the same business_id.

In [27]:
# Reviews of the first entry of the df_unprocessed dataframe
df_reviews[df_reviews['business_id'] == df_xunprocessed['business_id'][0]]

,business_id,text,user_id,relative_time_description,time_created,user_rating
0,RDE8pT0YZbz4BsO_sPKbcg,Great place for lunch! Pleasant atmosphere. Am...,Daniel Donnelly,2 months ago,2019-02-26 02:49:43,5
1,RDE8pT0YZbz4BsO_sPKbcg,It’s an amazing little coffee shop. The food ...,Valentina Hernandez,2 weeks ago,2019-04-11 19:31:10,5
2,RDE8pT0YZbz4BsO_sPKbcg,"Awesome coffee, delicious pastries, and an inc...",Patrick Hart,2 months ago,2019-02-19 17:40:37,5
3,RDE8pT0YZbz4BsO_sPKbcg,Gem in the Bronx! Their food and drinks are am...,Ange Mallonga,2 weeks ago,2019-04-15 17:20:13,5
4,RDE8pT0YZbz4BsO_sPKbcg,This place says “gentrification” all over it. ...,Nick Mennell,5 months ago,2018-11-29 15:50:10,5


Now we can generate features using the reviews metadata.

In [29]:
'''
df_ml = df_xunprocessed[['review_count','rating','price','std_of_stars',
                        'median_of_stars','stars_linear_coef','restaurant_density',
                        'restaurant_similar_density','user_ratings_total','zstar_all','zprice_all','transactions'
                        'is_chain','relative_user_ratings_total']]
'''
from datetime import datetime
from sklearn.linear_model import LinearRegression
import sys

regr = LinearRegression()

#df_unprocessed['reviews_per_week'] = np.nan
#df_unprocessed['oldest_review'] = np.nan # expressed in days from today
df_xunprocessed['std_of_stars'] = np.nan
df_xunprocessed['median_of_stars'] = np.nan
df_xunprocessed['stars_linear_coef'] = np.nan
#df_unprocessed['reactions_per_week'] = np.nan


# 
for i, restaurant in df_xunprocessed.iterrows():
    try:
        reviews = df_reviews[df_reviews['business_id'] == restaurant['business_id']]
        review_date=[]
    #print(len(reviews[reviews['user_rating']=='']))
    # difference between newest and oldest review in weeks
        for eachtime in reviews['time_created']:
            review_date.append(datetime.strptime(eachtime, "%Y-%m-%d %H:%M:%S"))
        dates = [d.toordinal() for d in review_date] # convert dates to integer
        dates = np.asarray(dates)
    # fit linear regression model to find coefficient (do stars increase or decrease?)
        regr.fit(dates.reshape(-1,1),reviews['user_rating'].values.reshape(-1,1)) 
    
    #df_unprocessed.loc[i,'reviews_per_week'] = nreviews / delta_time
    
    # The following way of calculating frequency is better. Since restaurants with a small number of reviews
    # that came at the same period would seem like they get reviews very frequently
    #df_unprocessed.loc[i,'reviews_per_week'] = nreviews / weeks_since_older_review
    #df_unprocessed.loc[i,'oldest_review'] = weeks_since_older_review
        #df_xunprocessed.loc[i,'std_of_stars'] = np.std(reviews['user_rating'])
        #df_xunprocessed.loc[i,'median_of_stars'] = np.median(reviews['user_rating'])
        df_xunprocessed.loc[i,'transactions_value'] = len(df_xunprocessed.loc[i,'transcations'])
        #print(len(df_unprocessed.loc[i,'transcations']))
    # same for this, use weeks_since_older review instead of delta_time
    #df_unprocessed.loc[i,'reactions_per_week'] = nreactions / weeks_since_older_review
        df_xunprocessed.loc[i,'stars_linear_coef'] = regr.coef_[0][0]
    
        if i % 100 == 0:
            print(i)
    except:
        print(restaurant['business_id'])
        print(sys.exc_info())
        continue

0
100
TfIvCgc2XgYqzyif-Cytqw
(<class 'ValueError'>, ValueError('Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.',), <traceback object at 0x7f127b4b6308>)
200
300
400
ISxL2HWJNCazHAdy5oblJA
(<class 'ValueError'>, ValueError('Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.',), <traceback object at 0x7f127b4bba88>)
500
600
hR1bG0jgd2_Z9Q5L6siTEw
(<class 'ValueError'>, ValueError('Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.',), <traceback object at 0x7f127b4b6fc8>)
700
jva4AMX7zRkNsY3DKX9LlQ
(<class 'ValueError'>, ValueError('Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.',), <traceback object at 0x7f127b4b6dc8>)
800
900
ePVlFbTzdVQFWie__kdplg
(<class 'ValueError'>, ValueError('Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.',), <traceback object at 0x7f127b4bbdc8>)
cF3deEEqK3Fhj22k0pE1_Q
(<class 'ValueError'>, ValueError('Found array wi

7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
6Bv2zPuLN1ni11TtCd5K4w
(<class 'ValueError'>, ValueError('Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.',), <traceback object at 0x7f127b4b6408>)
9400
9500
9600
9700
1J4qHPYqIH7iKzrZVkqUqQ
(<class 'ValueError'>, ValueError('Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.',), <traceback object at 0x7f127b4bbdc8>)
9800
9900
mBFnE5EDUwpZiEsSf_1piA
(<class 'ValueError'>, ValueError('Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.',), <traceback object at 0x7f127b4b7c48>)
10000
7F-PW3rS1Sp3dvg2gaAssw
(<class 'ValueError'>, ValueError('Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.',), <traceback object at 0x7f127b4babc8>)
3I2fkqd7SbtPAJiMJj4_vQ
(<class 'ValueError'>, ValueError('Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.',), <traceback object at 0x7f127b4c0488>)

Now we will make a dictionary of restaurant categories that will help us calculate relative quantities. The keys are the restaurant categories and the values are the indices of the restaurants that belong in each category.

In [30]:
# Create a dictionary of categories with the index of each corresponding restaurant
category_dic = {}
for i, restaurant in df_xunprocessed.iterrows():
    for category in restaurant['categories']:
        try:
            category_dic[category] += [i]
        except:
            category_dic[category] = [i]

In [31]:
# make price numeric
df_xunprocessed = df_xunprocessed.replace({'price':{'$':1,'$$':2,'$$$':3,'$$$$':4}})
#df_unprocessed = df_unprocessed.replace({'transcations':len({'[]'':0,'[pickup]':1, '[restaurant_reservation]':1, '[delivery]':1,'[pickup, restaurant_reservation]':2,'[restaurant_reservation, pickup]':2,'[pickup, delivery]':2,'[delivery, pick up]':2,'[restaurant_reservation, delivery':2,'pickup, delivery, restaurant_reservation':3,'pickup, restaurant_reservation, delivery':3})})
df_xunprocessed.head()

,business_id,categories,city,user_ratings_total,coordinates.latitude,coordinates.longitude,name,display_phone,location.display_address,location.state,...,is_closed,rating,review_count,price,transcations,url,std_of_stars,median_of_stars,stars_linear_coef,transactions_value
0,RDE8pT0YZbz4BsO_sPKbcg,"[cafes, coffee, catering]",South Bronx,108,40.810086,-73.921687,Mottley Kitchen,(929) 308-2099,"[402 E 140th St, South Bronx, NY 10454]",NY,...,False,4.5,63,2,"[pickup, delivery]",https://www.yelp.com/biz/mottley-kitchen-south...,0.000000,5.0,-0.000000,2.0
1,8G6H30Krmj8-OHs6hZfT1g,"[newamerican, southern, comfortfood]",New York,79,40.817300,-73.941710,Belle Harlem,(347) 819-4076,"[2363 Adam Clayton Powell Blvd, New York, NY 1...",NY,...,False,4.5,120,3,[],https://www.yelp.com/biz/belle-harlem-new-york...,0.000000,5.0,-0.000000,0.0
2,g_Zz87qghqq9bKHiKyws0w,"[tradamerican, speakeasies, latin]",Bronx,226,40.807370,-73.927520,Beatstro,(718) 489-9397,"[135 Alexander Ave, Bronx, NY 10454]",NY,...,False,4.0,151,2,"[pickup, restaurant_reservation, delivery]",https://www.yelp.com/biz/beatstro-bronx?adjust...,0.979796,5.0,0.063633,3.0
3,6AC4yhUdnh64zE6b5-n6OQ,[newamerican],New York,162,40.813340,-73.944670,Renaissance Harlem,(646) 838-7604,"[2245 Adam Clayton Powell, New York, NY 10027]",NY,...,False,4.0,144,2,"[restaurant_reservation, pickup, delivery]",https://www.yelp.com/biz/renaissance-harlem-ne...,0.000000,5.0,0.000000,3.0
4,W_0AX5JL3QZnX1oQNxQGtw,[mexican],Bronx,350,40.818565,-73.916828,Xochimilco Family Restaurant,(718) 402-5400,"[653 Melrose Ave, Bronx, NY 10455]",NY,...,False,4.5,76,2,[],https://www.yelp.com/biz/xochimilco-family-res...,1.600000,5.0,-0.029553,0.0


Here, we will calculate restaurant density within 1 mile radius and other relative quantities.  The z\* quantities are relative to the equivalent quantities of the surrounding restaurants.

Check if the new calculation for is_chain worked.

In [10]:
# this function returns distance in miles given longitude and latitude of two locations
import math
import sys
def distance(ilat,jlat,ilong,jlong):
    R = 6371.e3 # earth radius in m
    phi1 = math.radians(ilat)
    phi2 = math.radians(jlat)
    deltaphi = math.radians(jlat-ilat)
    deltalambda = math.radians(jlong-ilong)
    a = math.sin(deltaphi/2)**2 + math.cos(phi1)*math.cos(phi2) * math.sin(deltalambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c / 1609.34 # in miles
    return d

df_unprocessed['restaurant_density'] = np.nan
df_unprocessed['restaurant_similar_density'] = np.nan
df_unprocessed['zprice_all'] = np.nan
df_unprocessed['zreview_count_all'] = np.nan
df_unprocessed['zstar_all'] = np.nan
df_unprocessed['is_chain'] = np.nan
df_unprocessed['relative_user_ratings_total'] = np.nan

for i, restaurant in df_unprocessed.iterrows():
    try:
        price_all_list = []
        review_count_all_list = []
        review_per_week_all_list = []
        star_all_list = []
        density_similar_list = []
        density_all_list = []
        relative_user_ratings_list=0
    
        ilong = restaurant['coordinates.longitude']
        ilat = restaurant['coordinates.latitude']
        for category in restaurant['categories']:
            for jrestaurant in category_dic[category]:
                jlong = df_unprocessed.loc[jrestaurant]['coordinates.longitude']
                jlat = df_unprocessed.loc[jrestaurant]['coordinates.latitude']
                dist = distance(ilat,jlat,ilong,jlong)
            # if less than 1 mile (include self)
            # certain restaurants will be double counted if they belong to the same categories but
            # that's ok as it adds a weight for very similar restaurants
                if dist <= 1:
                    density_similar_list += [jrestaurant]
                    jrate = df_unprocessed.loc[jrestaurant]['user_ratings_total']
                    relative_user_ratings_list+=jrate
        df_unprocessed.loc[i,'restaurant_similar_density'] = len(density_similar_list)
        df_unprocessed.loc[i,'relative_user_ratings_total'] =(relative_user_ratings_list)/len(density_similar_list)
    
    
    # Loop over all resturants to calculate the "all" properties
        for j, jrestaurant in df_unprocessed.iterrows():
            jlong = jrestaurant['coordinates.longitude']
            jlat = jrestaurant['coordinates.latitude']
            dist = distance(ilat,jlat,ilong,jlong)
        # if less than 1 mile (include self in calculation of statistics)
            if dist <= 1:
                price_all_list += [jrestaurant['price']]
                review_count_all_list += [jrestaurant['review_count']]
            #review_per_week_all_list += [jrestaurant['reviews_per_week']]
                star_all_list += [jrestaurant['rating']]
                density_all_list += [j]
        
        
    # price and stars are not divided by standard deviation because it is often equal to 0
        df_unprocessed.loc[i,'zprice_all'] = (restaurant['price']-np.nanmean(price_all_list))/4.
    #df_unprocessed.loc[i,'relative_user_ratings_total'] = (restaurant['user_ratings_total']-np.nanmean(price_all_list))/4.
        df_unprocessed.loc[i,'zreview_count_all'] = (restaurant['review_count']-np.nanmean(review_count_all_list))/np.nanstd(review_count_all_list)
    #df_unprocessed.loc[i,'zreview_per_week_all'] = (restaurant['reviews_per_week']-np.nanmean(review_per_week_all_list))/np.nanstd(review_per_week_all_list)
        df_unprocessed.loc[i,'zstar_all'] = (restaurant['rating']-np.nanmean(star_all_list))/5.
        df_unprocessed.loc[i,'restaurant_density'] = len(density_all_list)
    #print(len(density_all_list))
        df_unprocessed.loc[i,'restaurant_similar_density'] = len(density_similar_list)
    # True if there are more than one
        df_unprocessed.loc[i,'is_chain'] = (len(df_unprocessed[df_unprocessed['name'] == restaurant['name']]) > 1)
    
        if i % 10 == 0:
            print(i)
        if i==10:
            print(df_unprocessed.head())
    except:
        print(restaurant['business_id'])
        print(sys.exc_info())
        continue

0
10
              business_id                            categories         city  \
0  RDE8pT0YZbz4BsO_sPKbcg             [cafes, coffee, catering]  South Bronx   
1  8G6H30Krmj8-OHs6hZfT1g  [newamerican, southern, comfortfood]     New York   
2  g_Zz87qghqq9bKHiKyws0w    [tradamerican, speakeasies, latin]        Bronx   
3  6AC4yhUdnh64zE6b5-n6OQ                         [newamerican]     New York   
4  W_0AX5JL3QZnX1oQNxQGtw                             [mexican]        Bronx   

   user_ratings_total  coordinates.latitude  coordinates.longitude  \
0                 108             40.810086             -73.921687   
1                  79             40.817300             -73.941710   
2                 226             40.807370             -73.927520   
3                 162             40.813340             -73.944670   
4                 350             40.818565             -73.916828   

                           name   display_phone  \
0               Mottley Kitchen  (929) 308

/opt/conda/envs/data3/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in double_scalars


1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2220
2230
2240
2250
2260
2270
2280
2290
2300
2310
2320
2330
2340
2350
2360
2370
2380
2390
2400
2410
2420
2430
2440
2450
2460
2470
2480
2490
2500
2510
2520
2530
2540
2630
2640
2650
2660
2670
2680
2690
2700
2710
2720
2730
2740
2750
2760
2770
2780
2790
2800
2810
2820
2830
2840
2850
2860
2870
2880
2890
2900
2910
2920
2930
2940
2950
2960
2970
2980
2990
3000
3010
3020
3030
3040
3050
3060
3070
3080
3090
3100
3110
3120
3130
3140
3150
3160
3170
3180
3190
3200
3210
3220
3230
3240
3250
3260
3270
3280
3290
3300
3310
3320
3330
3340
3350
3360
3370


14880
14890
14900
14910
14920
14930
14940
14950
14960
14970
14980
14990
15000
15010
15020
15030
15040
15050
15060
15070
15080
15090
15100
15110
15120
15130
15140
15150
15160
15170
15180
15190
15200
15210
15220
15230
15240
15250
15260
15270
15280
15290
15300
15310
15320
15330
15340
15350
15360
15370
15380
15390
15400
15410
15420
15430
15440
15450
15460
15470
15480
15490
15500
15510
15520
15530
15540
15550
15560
15570
15580
15590
15600
15610
15620
15630
15640
15650
15660
15670
15680
15690
15700
15710
15720
15730
15740
15750
15760
15770
15780
15790
15800
15810
15820
15830
15840
15850
15860
15870
15880
15890
15900
15910
15920
15930
15940
15950
15960
15970
15980
15990
16000
16010
16020
16030
16040
16050
16060
16070
16080
16090
16100
16110
16120
16130
16140
16150
16160
16170
16180
16190
16200
16210
16220
16230
16240
16250
16260
16270
16280
16290
16300
16310
16320
16330
16340
16350
16360
16370
16380
16390
16400
16410
16420
16430
16440
16450
16460
16470
16480
16490
16500
16510


In [11]:
df_unprocessed.head()
df_unprocessed[['name','is_chain']]

,name,is_chain
0,Mottley Kitchen,False
1,Belle Harlem,False
2,Beatstro,False
3,Renaissance Harlem,False
4,Xochimilco Family Restaurant,False
5,The Edge Harlem,False
6,Gun Hill Tavern,False
7,Charlie's Bar & Kitchen,False
8,Red Rooster Harlem,False
9,Franco's,False


In [32]:
dill.dump(df_xunprocessed,open('./data/NYC-features.dill','wb'))

In [13]:
len(category_dic)

260

In [5]:
len(df_unprocessed)

NameError: name 'df_unprocessed' is not defined